- Inspired from demo.ipynb in NGSIM.jl
- Takeaways
    - 101 has 5 segments
    - Start from bottom of screen, 1 is bottom of screen, 2 is exit ramp,
    3 is straight section, 4 is entry to road till entry ramp 
    join point, 5 is entry ramp

In [1]:
using NGSIM
using AutomotiveDrivingModels
using AutoViz
using Interact # For slider bar trajectory propagation

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/5dc1e185f8b2175037a0bb4bac5baa1c8ca39ea5-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/5dc1e185f8b2175037a0bb4bac5baa1c8ca39ea5-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
roadway = ROADWAY_101;

In [ ]:
length(scene) #108
fieldnames(scene) #n, entitities
typeof(scene.entities) # array of records
size(scene.entities) #500
typeof(scene.entities[1]) # Records.entity
fieldnames(scene.entities[1])

## Video making using recorded cars on ngsim
- Trajdatas is what the car trajectories are stored in
- We want to color the ego vehicle differently to be able to see it

In [ ]:
# argument 1 loads i101 7:50 to 8:05.
# load_trajdata function defined in NGSIM.jl/src/trajdata.jl
td1 = load_trajdata(1); 

scene = Scene(500)
egoid = 546

# Drive here in the notebook. Replay the trajectory as recorded in the ngsim data
@manipulate for i in 1000:2000
    temp_scene = get!(scene,td1,i)
    
    carcolors = Dict{Int,Colorant}()
    for veh in temp_scene
        #@show veh.id
        # if veh id matches the egoid color it blue otherwise green
        carcolors[veh.id] = 
        in(veh.id, egoid) ? colorant"blue" : colorant"green"
    end
    render(temp_scene, ROADWAY_101, 
        cam=CarFollowCamera{Int}(546,5.0),
#         cam=StaticCamera(VecE2(1966400, 570900), 5.0),
#         cam=FitToContentCamera(0.),
        car_colors=carcolors)
end

# Don't reinvent the wheel: Use existing environment
- For `observe!` from `Tim2DDriver` to work, had to make the following changes
    - Had to modify `AutomotiveDrivingModels.jl/src/2d/vehicles/scene_records.jl` to extend
the default length of the capacity in the record container from 100 to 300
    - Did the same capacity extension in `Records.jl/src/frames.jl`, `QueueRecords.jl` and 
`ListRecords.jl`
- Note that querying Tim2DDriver yields a `latLonAccel` return type. Had to create an array
to enable these values to work with `step`

In [92]:
using AutoEnvs

n_veh = 1
filepath = Pkg.dir("NGSIM", "data", "trajdata_i101_trajectories-0805am-0820am.txt")
params = Dict(
        "trajectory_filepaths"=>[filepath],
        "H"=>50,
        "primesteps"=>50,
        "n_veh"=>n_veh,
        "remove_ngsim_veh"=>false
)
# env = MultiagentNGSIMEnvVideoMaker(params)
env = MultiagentNGSIMEnv(params);
reset(env);

In [93]:
timestep = 0.1
model = Tim2DDriver(timestep,mlane = MOBIL(timestep));

In [94]:
# This will create a series of images in /tmp/episode1
# You can open the first and then press right arrow

# a = ones(n_veh, 2)
# a[2] = 0.0

for _ in 1:100
    render(env)
    observe!(model, env.scene, env.roadway, env.ego_vehs[1].id)
    latlonacc = rand(model)
    a = [latlonacc.a_lon latlonacc.a_lat]
    x, r, terminal, _ = step(env, a)
end

In [42]:
env.ego_vehs

1-element Array{Union{Records.Entity{AutomotiveDrivingModels.VehicleState,AutomotiveDrivingModels.VehicleDef,Int64}, Void},1}:
 Vehicle(697, VehicleState(VecSE2({1966761.174, 570570.356}, -0.717), Frenet(RoadIndex({751, 0.745057}, {1, 1}), 228.845, -1.515, -0.042), 17.041), VehicleDef(CAR, 4.724, 1.798))

In [ ]:
# Take a look at sisl/gail-driver/julia/envs/Auto2D.jl function reel_drive
# for video making inspiration

In [ ]:
# Need to figure out what input to give to get a sensible a from Tim2D
# Look at get_actions method within 
# AutomotiveDrivingModels/src/interface/simulation.jl to get inspiration
timestep = 0.1
model = Tim2DDriver(timestep,
        mlane = MOBIL(timestep))
# 
set_desired_speed!(model, 10.0)
reset(env)
get!(env.scene, env.trajdatas[env.traj_idx], env.t)
# for (i,veh) in enumerate(env.scene)
# #     env.scene, env.trajdatas[env.traj_idx], env.t
#     @show i    
#     @show veh
#     observe!(model, env.scene, env.roadway, veh.id)
        @show rand(model)
# end

# a = rand(model)

## Experiment with Tim driver
- Using the stadium2D demo from AutomotiveDrivingModels

In [ ]:
roadway = gen_stadium_roadway(3);

In [ ]:
scene = Scene()
push!(scene,Vehicle(VehicleState(VecSE2(10.0,-DEFAULT_LANE_WIDTH,0.0), roadway, 29.0), VehicleDef(), 1))
push!(scene,Vehicle(VehicleState(VecSE2(40.0,0.0,0.0), roadway, 22.0), VehicleDef(), 2))
push!(scene,Vehicle(VehicleState(VecSE2(70.0,-DEFAULT_LANE_WIDTH,0.0), roadway, 27.0), VehicleDef(), 3))

car_colors = get_pastel_car_colors(scene)
cam = FitToContentCamera()
render(scene, roadway, cam=cam, car_colors=car_colors);

In [ ]:
scene[1].id

In [ ]:
AutomotiveDrivingModels.observe!(model, scene, roadway, scene[1].id)

In [ ]:
a = rand(model)

In [ ]:
timestep = 0.1

model = Tim2DDriver(timestep,
        mlane = MOBIL(timestep),
    )
set_desired_speed!(model, 10.0)

for (i,veh) in enumerate(scene)
#     env.scene, env.trajdatas[env.traj_idx], env.t
        observe!(model, env.scene, env.roadway, veh.id)
        @show rand(model)
end

# nticks = 100
# rec = SceneRecord(nticks+1, timestep)
# simulate!(rec, scene, roadway, model, nticks)
# render(rec[0], roadway, cam=cam, car_colors=car_colors)